# Omdena Milan Covid19 and Crime

# Task 2 - Data Cleaning - Geodata

## Geodata

### For Colab - Uncomment as required

In [ ]:
# %pip install geopandas

In [ ]:

# from google.colab import drive
# drive.mount('/content/drive/')


### Load Files

In [ ]:
# reg_url = '/content/drive/MyDrive/Omdena_Milano_Covid_and_crime/Datasets/Task 1/Geojson/regions.geojson'
# reg_sim_url = '/content/drive/MyDrive/Omdena_Milano_Covid_and_crime/Datasets/Task 1/Geojson/regions_simplified.geojson'
# prov_url = '/content/drive/MyDrive/Omdena_Milano_Covid_and_crime/Datasets/Task 1/Geojson/provinces.geojson'
# prov_sim_url = '/content/drive/MyDrive/Omdena_Milano_Covid_and_crime/Datasets/Task 1/Geojson/provinces_simplified.geojson'
# munic_url = '/content/drive/MyDrive/Omdena_Milano_Covid_and_crime/Datasets/Task 1/Geojson/municipalities.geojson'
# munic_sim_url = '/content/drive/MyDrive/Omdena_Milano_Covid_and_crime/Datasets/Task 1/Geojson/municipalities_simplified.geojson'

## For using in GitHub locally

### Load Files

In [ ]:
reg_url = '../../../data/geo_data/regions.geojson'
reg_sim_url = '../../../data/geo_data/regions_simplified.geojson'
prov_url = '../../../data/geo_data/provinces.geojson'
prov_sim_url = '../../../data/geo_data/provinces_simplified.geojson'
munic_url = '../../../data/geo_data/municipalities.geojson'
munic_sim_url = '../../../data/geo_data/municipalities_simplified.geojson'

Load dependancies

In [ ]:

import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Polygon, MultiPolygon
import plotly.express as px
import plotly.graph_objects as go
import json
import folium as flm
from numba import jit, cuda

pd.set_option('display.max_columns', None)

### Read files and create a basic plot

In [ ]:
region = gpd.read_file(reg_url)
region.plot()
plt.show()

In [ ]:

prov = gpd.read_file(prov_url)
prov.plot()
plt.show()


In [ ]:
munics = gpd.read_file(munic_url)
munics.plot()
plt.show()

### Create `dataframes` using `Geopandas`

In [ ]:

reg_df = gpd.GeoDataFrame(region)
reg_df


In [ ]:
prov_df = gpd.GeoDataFrame(prov)
prov_df

In [ ]:

munic_df = gpd.GeoDataFrame(munics)
munic_df

### Create Longitude and Latitude columns using `Shapely`

In [ ]:
print(reg_df.crs)

In [ ]:
# reg_df = reg_df.to_crs('EPSG:3857')
reg_df['region_lon'] = reg_df['geometry'].centroid.x
reg_df['region_lat'] = reg_df['geometry'].centroid.y
# reg_df = reg_df.to_crs('EPSG:4326')

In [ ]:
reg_df

In [ ]:
# prov_df = prov_df.to_crs('EPSG:3857')
prov_df['region_lon'] = prov_df['geometry'].centroid.x
prov_df['region_lat'] = prov_df['geometry'].centroid.y
# prov_df = prov_df.to_crs('EPSG:4326')

In [ ]:
prov_df

In [ ]:
# munic_df = munic_df.to_crs('EPSG:3857')
munic_df['region_lon'] = munic_df['geometry'].centroid.x
munic_df['region_lat'] = munic_df['geometry'].centroid.y
# munic_df = munic_df.to_crs('EPSG:4326')

In [ ]:
munic_df

In [ ]:
with open(reg_url) as response:
    regs_m = json.load(response)

regs_m["features"][0]

In [ ]:
region

In [ ]:
with open(reg_sim_url) as f:
    geojson_reg = json.load(f)

df = pd.json_normalize(geojson_reg['features'])
df['region_name'] = df['properties.reg_name']

fig = px.choropleth_mapbox(df, geojson=geojson_reg, featureidkey='properties.reg_istat_code',
                           locations='properties.reg_istat_code', color='region_name',
                           color_discrete_sequence=px.colors.qualitative.Dark24, center={'lon': 12.5, 'lat': 41.9},
                           mapbox_style='carto-positron', zoom=4.5, opacity=0.8)
fig.update_layout(margin={'r':0,'t':0,'l':0,'b':0})
fig.show()

In [ ]:
with open(reg_sim_url) as f:
        geojson_reg = json.load(f)

In [ ]:
def regions(data):
    df = pd.json_normalize(data['features'])
    df['region_name'] = df['properties.reg_name']

    df['region_lon'] = reg_df['region_lon']
    df['region_lat'] = reg_df['region_lat']

    # Define the markers
    markers = go.Scattermapbox(
        lat=df['region_lat'],
        lon=df['region_lon'],
        mode='markers',
        marker=go.scattermapbox.Marker(
            size=10,
            color='yellow'
        ),
        text=df[['region_name', 'properties.reg_istat_code']]
    )

    # Create the choropleth map
    fig = px.choropleth_mapbox(df, geojson=geojson_reg, featureidkey='properties.reg_istat_code',
                        locations='properties.reg_istat_code', color='region_name',
                        color_discrete_sequence=px.colors.qualitative.Dark24, center={'lon': 12.5, 'lat': 41.9},
                        mapbox_style='carto-positron', zoom=4.5, opacity=0.8)

    # Add the markers to the figure
    fig.add_trace(markers)

    # Update the layout
    fig.update_layout(margin={'r':0,'t':0,'l':0,'b':0})

    # Show the figure
    fig.show()

In [ ]:
regions(geojson_reg)

In [ ]:
with open(prov_sim_url) as f:
    geojson_prov = json.load(f)

In [ ]:
def provs(data):
    df2 = pd.json_normalize(data['features'])
    df2['prov_name'] = df2['properties.prov_name']

    fig = px.choropleth_mapbox(df2, geojson=data, featureidkey='properties.prov_istat_code_num',
                            locations='properties.prov_istat_code_num', color='prov_name',
                            color_discrete_sequence=px.colors.qualitative.Dark24, center={'lon': 12.5, 'lat': 41.9},
                            mapbox_style='carto-positron', zoom=4.5, opacity=0.8)
    fig.update_layout(margin={'r':0,'t':0,'l':0,'b':0})
    fig.show()

In [ ]:
provs(geojson_prov)

In [ ]:
with open(munic_sim_url) as f:
    geojson_munic = json.load(f)

In [ ]:
df6 = pd.json_normalize(geojson_munic['features'])
df6.head(1)

In [ ]:
m = flm.Map(location=[42.819748, 12.772184], zoom_start=6)

tooltip = flm.GeoJsonTooltip(fields=["properties.reg_name"], labels=True)

flm.Choropleth(
geo_data=geojson_reg,
name="Italy Regions",
data=geojson_reg,
columns=["reg_name", "reg_istat_code"],
key_on="properties.reg_name",
fill_color="Reds",
fill_opacity=0.7,
line_opacity=0.2,
legend_name="ISTAT Numbers",
tooltip=tooltip,
).add_to(m)

flm.GeoJsonTooltip(['name', 'ISTAT Code']).add_to(cp.geojson)

flm.LayerControl().add_to(m)

m

In [ ]:
with open(prov_url) as response:
    provs_m = json.load(response)

provs_m["features"][0]

In [ ]:
print(prov_df["prov_name"][2])
print(provs_m["features"][0]["properties"]['prov_name'])

In [ ]:
print(reg_df["reg_name"][2])
print(regs_m["features"][2]["properties"]['reg_name'])